In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python

In [3]:
import sys
sys.path.append(r"./python")

import os
import json
from model import *

#根据时间情况修改index和language值
index =  "wy_demo_excel"
# index =  "wy_demo_wts"
embedding_endpoint_name = "cohere.embed-multilingual-v3"

embedding_type = 'bedrock' if embedding_endpoint_name.find('titan') or embedding_endpoint_name.find('cohere') else 'sagemaker'
embeddings = init_embeddings_bedrock(embedding_endpoint_name)

In [ ]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
from opensearch_multimodel_dataload import add_multimodel_documents
text_max_length = 2000
import os
import json

excel_file_path = '/home/ec2-user/SageMaker/wiwynn/EXCEL/'
files = os.listdir(excel_file_path)
print('Total file number:',len(files))

texts = []
metadatas = []
for file in files:
    print('file:',file)
    if file.split('.')[-1] == 'json':
        with open(excel_file_path+file, 'r') as excel_file:
            data = json.load(excel_file)

            error_code = data['Error code'] if data['Error code'] is not None else ""
            failure_symptom = data['Failure symptom'] if data['Failure symptom'] is not None else ""
            suggested_action = data['Suggested action'] if data['Suggested action'] is not None else ""
            real_action = data['Real action'] if data['Real action'] is not None else ""
            error_log = data['Error logs'] if data['Error logs'] is not None else ""
            symptom_code = data['Symptom code'] if data['Symptom code'] is not None else ""

            sentence = ''
            if len(symptom_code) > 0:
                sentence = error_code + "," + symptom_code + "," + error_log
            else:
                sentence = error_code + "," + error_log

            if real_action is not None and suggested_action is not None:
                text = 'real_action:' + real_action + '; suggested_action:' + suggested_action
            elif real_action is not None:
                text = 'real_action:' + real_action
            elif suggested_action is not None:
                text = 'suggested_action:' + suggested_action
            texts.append(text)

            metadata = {}
            metadata['error_code'] = error_code
            metadata['failure_symptom'] = failure_symptom
            metadata['suggested_action'] = suggested_action
            metadata['real_action'] = real_action
            metadata['error_log'] = error_log
            metadata['symptom_code'] = symptom_code
            metadata['sentence'] = sentence[:text_max_length]
            metadatas.append(metadata)

if len(metadatas) > 0:
    text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
    add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas
        )
    print('finish save in vectore store:',index)

In [8]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
from opensearch_multimodel_dataload import add_multimodel_documents
import csv
text_max_length = 2000

excel_file_path = '../docs/wy/output_excel.csv'

with open(excel_file_path,'r') as excel_file:
    lines = csv.reader(excel_file)
    i = 0
    texts = []
    metadatas = []
    for line in lines:
        i += 1
        if i == 1:
            continue
        
        error_code = line[0]
        failure_symptom = line[1]
        suggested_action = line[2]
        real_action = line[3]
        error_log = line[4]
        symptom_code = line[5]
        
        sentence = ''
        if len(symptom_code) > 0:
            sentence = error_code + "," + symptom_code + "," + error_log
        else:
            sentence = error_code + "," + error_log
            
        text = 'real_action:' + real_action + '; suggested_action:' + suggested_action
        texts.append(text)
        
        metadata = {}
        metadata['error_code'] = error_code
        metadata['failure_symptom'] = failure_symptom
        metadata['suggested_action'] = suggested_action
        metadata['real_action'] = real_action
        metadata['error_log'] = error_log
        metadata['symptom_code'] = symptom_code
        metadata['sentence'] = sentence[:text_max_length]
        metadatas.append(metadata)

    text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
    add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas
        )
    print('finish save in vectore store:',index)

finish save in vectore store: wy_demo_excel_0822


In [10]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
from opensearch_multimodel_dataload import add_multimodel_documents
import csv
text_max_length = 2000

excel_file_path = '../docs/wy/output_WTS.csv'

with open(excel_file_path,'r') as excel_file:
    lines = csv.reader(excel_file)
    i = 0
    texts = []
    metadatas = []
    for line in lines:
        i += 1
        if i == 1:
            continue
        
        wts_id = line[0]
        teamProject = line[1]
        error_message = line[2]
        error_code = line[3]
        symptom_code = line[4]
        error_log = line[5]
        solution = line[6]
        
        sentence = ''
        if len(symptom_code) > 0:
            sentence = error_code + "," + symptom_code + "," + error_log
        else:
            sentence = error_code + "," + error_log
            
        text = 'id:' + wts_id + '; System.TeamProject:' + teamProject
        texts.append(text)
        
        metadata = {}
        metadata['id'] = wts_id
        metadata['teamProject'] = teamProject
        metadata['error_message'] = error_message
        metadata['error_code'] = error_code
        metadata['symptom_code'] = symptom_code
        metadata['error_log'] = error_log
        metadata['solution'] = solution
        metadata['sentence'] = sentence[:text_max_length]
        metadatas.append(metadata)

    text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
    add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas
        )
    print('finish save in vectore store:',index)

finish save in vectore store: wy_demo_wts_0822
